In [1]:
import pandas as pd

In [2]:
path='/mnt/big/data/ds/nlp/tweets/'

In [3]:
def load_tweets(path):
    return (pd.read_csv(path,sep=';',
                          header=None,                          
                          names=('id','time','author','msg','sent','p1','p2','p3','p4','p5','p6','p7')))

In [4]:
dfp=load_tweets(path+'positive.csv')
dfn=load_tweets(path+'negative.csv')
df=pd.concat([dfp,dfn])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226834 entries, 0 to 111922
Data columns (total 12 columns):
id        226834 non-null int64
time      226834 non-null int64
author    226834 non-null object
msg       226834 non-null object
sent      226834 non-null int64
p1        226834 non-null int64
p2        226834 non-null int64
p3        226834 non-null int64
p4        226834 non-null int64
p5        226834 non-null int64
p6        226834 non-null int64
p7        226834 non-null int64
dtypes: int64(10), object(2)
memory usage: 22.5+ MB


In [6]:
dfn.head(5)

,id,time,author,msg,sent,p1,p2,p3,p4,p5,p6,p7
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [7]:
dfp.head(5)

,id,time,author,msg,sent,p1,p2,p3,p4,p5,p6,p7
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [8]:
import pymorphy2
from nltk.tokenize import word_tokenize
class MorphProvider:
    def __init__(self):
        self.cache = {}
        self.morph = pymorphy2.MorphAnalyzer()
    
    def __call__(self, w):
        w = w.lower()
        cached = self.cache.get(w)
        if cached:
            return cached
        try:
            morphed = self.morph.parse(w)[0].normal_form
            self.cache[w] = morphed
            return morphed
        except:
            return None
    
    def morph_string(self, s):
        words = word_tokenize(s)
        return " ".join([self.__call__(w) for w in words])  
morph = MorphProvider()  

In [9]:
from tqdm import tqdm

In [10]:
#df["norm"] = [morph.morph_string(t) for t in tqdm(df.msg)]

In [11]:
#df.norm.head(5)

In [12]:
import sklearn.feature_extraction.text

In [13]:
vctr = sklearn.feature_extraction.text.CountVectorizer(lowercase=True,max_features=1024*64,
                                                       token_pattern="[а-яА-Яa-zA-Z]{3,20}",max_df=0.100,min_df=0.000001)
vecs = vctr.fit_transform(df.msg)
vecs.shape

(226834, 65536)

In [14]:
list(vctr.vocabulary_)[:20]

['плеча',
 'лиама',
 'nopizozywory',
 'тихо',
 'фантазией',
 'гоген',
 'дописывать',
 'колонку',
 'ссылке',
 'moreee',
 'driblet',
 'заплела',
 'родов',
 'базарит',
 'lielielieliee',
 'rustlingbehind',
 'тишина',
 'здоровой',
 'evgenmatreshkin',
 'туфлях']

In [15]:
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
def l_model():
    return lgb.LGBMClassifier(
                boosting_type='gbdt', 
                num_leaves=128, 
                max_depth=4, 
                learning_rate=0.1, 
                n_estimators=300, 
                max_bin=255, 
                subsample_for_bin=50000, 
                objective='binary', 
                #eval_metric='binary_logloss',
                min_split_gain=0, 
                min_child_weight=5, 
                min_child_samples=10, 
                subsample=1, 
                subsample_freq=1, 
                colsample_bytree=1, 
                reg_alpha=0, 
                reg_lambda=0, 
                seed=0, 
                nthread=4, 
                device='cpu',
                silent=False)

In [16]:
import sklearn.pipeline
import sklearn.naive_bayes
text_clf = sklearn.pipeline.Pipeline([('vect', vctr),
                     #('clf', l_model())])
                     ('clf', sklearn.naive_bayes.MultinomialNB())])

In [17]:
from sklearn.model_selection import train_test_split
import numpy as np

In [18]:
X_train, X_test, y_train, y_test = train_test_split(df.msg, (df.sent==1), test_size=0.2, random_state=42)

In [19]:
text_clf.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.1, max_features=65536, min_df=1e-06,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='[а-яА-Яa-zA-Z]{3,20}',
        tokenizer=None, vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [20]:
predicted = text_clf.predict(X_test)

In [21]:
import sklearn.metrics
print(sklearn.metrics.classification_report(y_test, predicted,
     target_names=['negative','positive']))

             precision    recall  f1-score   support

   negative       0.73      0.76      0.74     22480
   positive       0.75      0.72      0.74     22887

avg / total       0.74      0.74      0.74     45367



In [22]:
sklearn.metrics.confusion_matrix(y_test, predicted)

array([[16985,  5495],
       [ 6320, 16567]])